In [23]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

import pysal 
import libpysal 

import oracledb

In [61]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection=oracledb.connect(
     user="username",
     password="password",
     dsn=dsn)

def g(df):
  return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

cur = connection.cursor()

query = 'SELECT * FROM COUNTRY'

countries = pd.read_sql(query, con=connection)
countries = g(countries.copy())

query = 'SELECT * FROM CLIMATEAGREEMENT'

climate_agreements = pd.read_sql(query, con=connection)
climate_agreements = g(climate_agreements.copy())

query = 'SELECT * FROM CLIMATEDATA'

esg = pd.read_sql(query, con=connection)
esg = g(esg.copy())

cur.close()


/var/folders/vf/y8fvcljn6g79p5yz6w4fc4wh0000gn/T/ipykernel_52464/944537674.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countries = pd.read_sql(query, con=connection)
/var/folders/vf/y8fvcljn6g79p5yz6w4fc4wh0000gn/T/ipykernel_52464/944537674.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  climate_agreements = pd.read_sql(query, con=connection)
/var/folders/vf/y8fvcljn6g79p5yz6w4fc4wh0000gn/T/ipykernel_52464/944537674.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  esg = pd.read_sql(query, con=connection)


### As much as we would like to do causal analysis, this is not possible since we don't have any effective control group. As such, we will focus on correlational analysis of countries and the extent to which they respond to cliamte agreements

# CLEANING

In [62]:
# climate_agreements = pd.read_excel("/Users/silas/Downloads/ClimateAgreement_Data_RAW.xlsx")
climate_agreements.head()

,COUNTRYCODE,COUNTRYNAME,AGREEMENTNAME,DATESIGNED,TARGET,TARGETYEAR
0,BRA,Brazil,Paris,2023-11-03,2025 net GHG emissions limit of 1.32 GtCO2e (4...,2030
1,BRA,Brazil,Net Zero,NaT,Climate neutral by 2050,2050
2,CAN,Canada,Paris,2021-07-12,At least 40-45% below 2005 levels by 2030,2030
3,CAN,Canada,Net Zero,NaT,Net zero GHG emissions by 2050,2050
4,CHL,Chile,Paris,2020-04-01,Chile commits to a GHG emission budget not exc...,2030


In [64]:
unique_cnames = climate_agreements['COUNTRYNAME'].unique().tolist()
print(unique_cnames)

['Brazil', 'Canada', 'Chile', 'China', 'Ethiopia', 'Germany', 'India', 'Indonesia', 'Japan', 'Kazakhstan', 'Kenya', 'Colombia', 'Costa-rica', 'Argentina', 'Australia', 'Bhutan', 'Mexico', 'Morocco', 'Nepal', 'Nigeria', 'Norway', 'Peru', 'Philippines', 'Singapore', 'Switzerland', 'Thailand', 'Turkey', 'Vietnam']


In [65]:
# countries = pd.read_excel("/Users/silas/Downloads/Country_Data_RAW.xlsx")
# columns_to_drop = ['Unnamed: 0']
# countries = countries.drop(columns=columns_to_drop)

new_column_names = {'COUNTRYCODE':'country_code', 'COUNTRYNAME': 'country_name', 'CAPITALCITY': 'capital_city', 'REGION': 'region', 'INCOMEGROUP': 'income_group', 'GOVERNMENTEFFECTIVENESS': 'gov_effectiveness', 'POPULATIONDENSITY':'pop_density', 'DEVELOPEDORDEVELOPING':'econ_status', 'TRADEBLOC':'group'}
countries = countries.rename(columns=new_column_names)
countries.head()

,country_code,country_name,region,capital_city,income_group,gov_effectiveness,pop_density,econ_status,group
0,BMU,Bermuda,North America,Hamilton,High income: nonOECD,NaN,NaN,Developed,None
1,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN
2,BRA,Brazil,Latin America & Caribbean,Brasilia,Upper middle income,-0.588,25.643,Developing,USAN
3,BRB,Barbados,Latin America & Caribbean,Bridgetown,High income: nonOECD,0.429,653.953,Developing,CARICOM
4,BRN,Brunei Darussalam,East Asia & Pacific,Bandar Seri Begawan,High income: nonOECD,1.422,84.511,Developing,None


In [66]:
# esg = pd.read_excel("/Users/silas/Downloads/Climate_Data_RAW.xlsx")
# columns_to_drop = ['Unnamed: 0']
# esg = esg.drop(columns=columns_to_drop)

new_column_names = {'COUNTRYCODE': 'country_code', 'YEAR': 'year', 'RENEWABLEENERGYTARGET': 'target', 'RENEWABLEENERGYOUTPUT':'renewable_nrg_output', 'RENEWABLEENERGYCONSUMPTION':'renewable_nrg_consump', 'CO2': 'co2', 'GINI':'gini', 'TREECOVERLOSS':'tree_loss'}
esg = esg.rename(columns=new_column_names)
esg.head()

,country_code,year,target,renewable_nrg_output,renewable_nrg_consump,co2,gini,tree_loss
0,BLR,2009,No,0.35,7.39,6.0158,27.7,35543.0
1,BEL,2009,Yes,6.06,4.87,9.2534,28.6,5176.0
2,BLZ,2009,No,79.29,29.18,1.6933,NaN,10264.0
3,BEN,2009,No,0.78,52.83,0.4816,NaN,4929.0
4,BTN,2009,No,100.00,92.38,0.5688,NaN,1175.0


In [67]:
complete = pd.merge(countries, esg, on='country_code')
complete.head()

,country_code,country_name,region,capital_city,income_group,gov_effectiveness,pop_density,econ_status,group,year,target,renewable_nrg_output,renewable_nrg_consump,co2,gini,tree_loss
0,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2009,No,39.49,20.62,1.3501,49.2,225270.0
1,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2010,No,33.95,19.93,1.4381,NaN,451759.0
2,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2012,No,32.31,17.36,1.6047,46.6,273544.0
3,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2013,No,32.18,16.02,1.6870,47.6,179823.0
4,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2011,No,34.20,18.10,1.5382,46.1,330239.0


In [68]:
original_list = unique_cnames
rename_mapping = {'Costa-rica': 'Costa Rica'}
unique_cnames = [rename_mapping[item] if item in rename_mapping else item for item in original_list]
print(unique_cnames)

['Brazil', 'Canada', 'Chile', 'China', 'Ethiopia', 'Germany', 'India', 'Indonesia', 'Japan', 'Kazakhstan', 'Kenya', 'Colombia', 'Costa Rica', 'Argentina', 'Australia', 'Bhutan', 'Mexico', 'Morocco', 'Nepal', 'Nigeria', 'Norway', 'Peru', 'Philippines', 'Singapore', 'Switzerland', 'Thailand', 'Turkey', 'Vietnam']


In [69]:
complete_agreement_countries = complete[complete['country_name'].isin(unique_cnames)]
complete_agreement_countries

,country_code,country_name,region,capital_city,income_group,gov_effectiveness,pop_density,econ_status,group,year,target,renewable_nrg_output,renewable_nrg_consump,co2,gini,tree_loss
16,BRA,Brazil,Latin America & Caribbean,Brasilia,Upper middle income,-0.588,25.643,Developing,USAN,2009,Yes,89.00,48.92,1.7993,53.7,1817901.0
17,BRA,Brazil,Latin America & Caribbean,Brasilia,Upper middle income,-0.588,25.643,Developing,USAN,2010,Yes,84.72,46.81,2.0266,NaN,2688896.0
18,BRA,Brazil,Latin America & Caribbean,Brasilia,Upper middle income,-0.588,25.643,Developing,USAN,2012,Yes,82.45,43.49,2.2714,53.4,2918633.0
19,BRA,Brazil,Latin America & Caribbean,Brasilia,Upper middle income,-0.588,25.643,Developing,USAN,2013,Yes,76.69,42.32,2.4134,52.7,1945506.0
20,BRA,Brazil,Latin America & Caribbean,Brasilia,Upper middle income,-0.588,25.643,Developing,USAN,2011,Yes,87.12,45.26,2.1106,52.9,1923364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,VNM,Vietnam,East Asia & Pacific,Hanoi,Lower middle income,0.175,310.973,Developing,ASEAN,2017,Yes,NaN,28.52,2.4446,NaN,286940.0
2908,VNM,Vietnam,East Asia & Pacific,Hanoi,Lower middle income,0.175,310.973,Developing,ASEAN,2018,Yes,NaN,24.59,3.0147,35.7,225155.0
2909,VNM,Vietnam,East Asia & Pacific,Hanoi,Lower middle income,0.175,310.973,Developing,ASEAN,2019,Yes,NaN,20.65,3.5678,NaN,216779.0
2910,VNM,Vietnam,East Asia & Pacific,Hanoi,Lower middle income,0.175,310.973,Developing,ASEAN,2022,Yes,NaN,NaN,NaN,NaN,NaN


In [70]:
complete_agreement_countries = complete_agreement_countries[(complete_agreement_countries['year'] >= 2012) & (complete_agreement_countries['year'] <= 2020)]
complete_agreement_countries = complete_agreement_countries.copy()  # Create a copy of the DataFrame slice
complete_agreement_countries['year'] = complete_agreement_countries['year'].astype(float)

developed = complete_agreement_countries[complete_agreement_countries['econ_status'] == 'Developed'].copy()
print(developed.shape)

developing = complete_agreement_countries[complete_agreement_countries['econ_status'] == 'Developing'].copy()
print(developing.shape)

(63, 16)
(189, 16)


In [71]:
nearest = alt.selection_point(nearest=True, on='mouseover', fields=['year'])

line = alt.Chart(complete_agreement_countries).mark_line(strokeWidth=3).encode(
    alt.X('year:O', title='Year'),
    y='co2',
    color=alt.Color('country_name:N', legend=alt.Legend(title='Country'), scale=alt.Scale(scheme='category20'))  # Use category20 color scheme
)

selectors = alt.Chart(complete_agreement_countries).mark_point().encode(
    alt.X('year:O', title='Year'),
    opacity=alt.value(0)
)

selectors_near = selectors.encode(
    opacity=alt.condition(nearest, alt.value(0.5), alt.value(0.3))
).add_params(nearest)

points = line.mark_circle().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
    tooltip=['country_name']
).add_params(nearest)

text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'co2:Q', alt.value(' '))
)

rules = alt.Chart(complete_agreement_countries).mark_rule(color='gray').encode(
    x=alt.X('year:O', title='Year', axis=alt.Axis(labelAngle=-25))
).transform_filter(
    nearest
)

layers = alt.layer(
    line, selectors, points, text, rules,
).properties(
    width=1100, height=700,
    title="Fig 1. Total CO2 emissions for Paris Climate Accord Countries from 2012 to 2020"
).configure_title(
    fontSize=30
).configure_legend(
    symbolSize=400 
).configure_text(fontSize=15)

chart = layers.configure_axis(
    labelFontSize=15 
)
chart
chart


alt.LayerChart(...)

### This line chart over time before and after the time of the policy (2016) gives us a rough idea of a before-after comparison  of the CO2 emissions for the countries that signed the Climate Accord. We also get an idea of the highest CO2 emitting countries, which appear to be Australia, Canada, and Kazakstan. More details of specific values for each year can be found by hovering over the graph. Overall, we get the impression that prior to the policy and after the policy, most countries either maintain CO2 levels or decrease them slightly over time.

In [72]:
df_2016 = complete_agreement_countries[complete_agreement_countries['year'] == 2016]
df_2020 = complete_agreement_countries[complete_agreement_countries['year'] == 2020]

comp_df = pd.concat([df_2016, df_2020])

selector = alt.selection_point(fields = ['country_name'])

base = alt.Chart(comp_df).properties(
    width=300,
    height=250,
).add_params(selector).configure_axis(
    labelFontSize=14, 
    titleFontSize=30  
)

chart = base.mark_bar().encode(
    x=alt.X('country_name:N', title='Country', sort='-y', axis=alt.Axis(labelAngle=-20)),
    y='co2:Q',
    column='year:N', 
    color='year:N',
    opacity=alt.condition(selector, alt.value(1), alt.value(0.1)),
    tooltip=['country_name', 'co2', 'year']
).properties(
    width=900,
    height=600,
).properties(
    title = "Fig 2. Absolute CO2 Levels for 2016 and 2020 for Paris Climate Accord Countries"
).configure_title(
    fontSize=30
).configure_axis(
    labelFontSize=11 
)
chart

alt.Chart(...)

### This bar chart allows us to isolate specific countries before and after the time of the policy (2016) and gives us a rough idea of a before-after comparison  of the CO2 emissions for the countries that signed the Climate Accord. We also get an idea of the highest CO2 emitting countries in absolute terms, which appear to be Australia, Canada, and Kazakstan. More details of specific values for each year can be found by hovering over the graph. However, looking at changes in absolute terms is not fair for smaller countries since they emit less CO2 to begin with from year to year and vary widely with the population and energy consumption of the population. As such, the optimal metric for changes in CO2 emissions would be % changes since it is proportional each countries CO2 emissions.

In [73]:
complete_agreement_countries_2016 = complete_agreement_countries[complete_agreement_countries['year'] == 2016]
complete_agreement_countries_2020 = complete_agreement_countries[complete_agreement_countries['year'] == 2020]

merged_df = pd.merge(complete_agreement_countries_2016, complete_agreement_countries_2020, on=['country_name', 'country_code'], suffixes=('_2016', '_2020'))

merged_df['co2_percentage_change'] = ((merged_df['co2_2020'] - merged_df['co2_2016']) / merged_df['co2_2016']) * 100

result_df = merged_df[['country_name', 'country_code', 'co2_percentage_change']]

chart = alt.Chart(result_df).mark_bar().encode(
    x=alt.X('country_name:N', title='Country', sort='y', axis=alt.Axis(labelAngle=-20)),  # Sort countries by co2_difference in descending order
    y=alt.Y('co2_percentage_change:Q', title='% Change in CO2 from 2016 to 2020'),
    tooltip=['country_name', 'co2_percentage_change'],
    color=alt.condition(
        alt.datum.co2_percentage_change > 0,
        alt.value('red'),  # Positive difference
        alt.value('green')  # Negative difference
    )
).properties(
    width=1000,
    height=500,
    title='Fig 3. % Change in CO2 Emissions (2016-2020) for Countries Post-Paris Climate Accord'
)

horizontal_line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='black').encode(y='y')

combined_chart = (chart + horizontal_line).configure_title(
    fontSize=25  
).configure_axis(
    labelFontSize=12  
)

combined_chart

alt.LayerChart(...)

### A simple difference in means chart between emissions in 2020 and 2016 shows the countries that appear to have made the most progress on their CO2 emissions. We see that Peru, Mexico, and Germany all appear to have made the largest changes in their CO2 emissions proportional to their 2016 levels. Conversely, Although Vietnam, Nepal, and Indonesia signed onto the Paris Climate Agreement with the intention of reducing their CO2 emissions, but their CO2 emissions have actually increased realtive to their 2016 levels! With this simple analysis, we find that these countries did not follow through with their commitment to the Paris Climate Agreement.

In [74]:
merged_df = result_df.merge(complete_agreement_countries, on='country_name', how='left')
merged_df = merged_df[merged_df['year']== 2016]

chart = alt.Chart(merged_df).mark_circle(size = 400).encode(
    alt.X('gov_effectiveness:Q', title = "Government Effectiveness"),
    alt.Y('co2_percentage_change:Q', title = "% Change in CO2 from 2016 to 2020"),				
    alt.Color('region:N'),
    alt.Tooltip('country_name:N')
).properties(
    width=800, height=600,
    title = "Fig 4. Scatterplot of CO2 emissions vs. Government Effectiveness"
).configure_title(
    fontSize=25
).configure_axis(
    labelFontSize=15  
)
chart

alt.Chart(...)

### Here we get an idea of the relationship between government effectiveness and the % change in CO2 emissions post-treaty. We see that countries with high government effectiveness (above 1.0 on this scale) also appear to have negative % changes in CO2 emissions post-treaty. This makes sense since governments are important for facilitating the reduction in CO2 emissions in the private sector: governments with high effectiveness more competently achieve their government objectives despite private interests in using existing technology that generates CO2 emissions. Conversely, governments with low effectiveness are less able to consistently create change in the private sector and transition away from CO2-emitting production.

# The next portion of our analysis involves a more rigorous approach of seeing how treaty-signing countries compare to non-treaty signing countries. This is an important counterfactual study since we can see if signing the treaty has an substantial correlation with a reduction in CO2 emmissions. Anyway, if even countries which don't sign the treaty also reduce their CO2 emissions, this might imply that treaty-signing countries are on-average not any better than non-treaty-singing countries!

In [75]:
rest_of_the_world = complete[~complete['country_name'].isin(unique_cnames)]
rest_of_the_world

,country_code,country_name,region,capital_city,income_group,gov_effectiveness,pop_density,econ_status,group,year,target,renewable_nrg_output,renewable_nrg_consump,co2,gini,tree_loss
0,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2009,No,39.49,20.62,1.3501,49.2,225270.0
1,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2010,No,33.95,19.93,1.4381,NaN,451759.0
2,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2012,No,32.31,17.36,1.6047,46.6,273544.0
3,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2013,No,32.18,16.02,1.6870,47.6,179823.0
4,BOL,Bolivia,Latin America & Caribbean,La Paz,Lower middle income,-0.564,11.151,Developing,USAN,2011,No,34.20,18.10,1.5382,46.1,330239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,ZWE,Zimbabwe,Sub-Saharan Africa,Harare,Low income,-1.255,41.343,Developing,None,2017,No,NaN,82.63,0.6631,44.3,NaN
3004,ZWE,Zimbabwe,Sub-Saharan Africa,Harare,Low income,-1.255,41.343,Developing,None,2018,No,NaN,80.43,0.7354,NaN,NaN
3005,ZWE,Zimbabwe,Sub-Saharan Africa,Harare,Low income,-1.255,41.343,Developing,None,2019,No,NaN,81.52,0.6633,50.3,NaN
3006,ZWE,Zimbabwe,Sub-Saharan Africa,Harare,Low income,-1.255,41.343,Developing,None,2022,No,NaN,NaN,NaN,NaN,NaN


In [76]:
#Treaty values
complete_agreement_countries_post = complete_agreement_countries 
complete_agreement_countries_post = complete_agreement_countries_post.loc[:, ['country_name', 'year', 'co2', 'pop_density', 'gov_effectiveness']]

complete_agreement_countries_post = complete_agreement_countries_post[(complete_agreement_countries_post['year'] >= 2016) & (complete_agreement_countries_post['year'] <= 2020)].copy()
complete_agreement_countries_post['year'] = complete_agreement_countries_post['year'].astype(int)

complete_agreement_countries_post = complete_agreement_countries_post.dropna(subset=['co2'])
complete_agreement_countries_post['year'] = pd.to_numeric(complete_agreement_countries_post['year'])
complete_agreement_countries_post
complete_agreement_countries_post = sm.add_constant(complete_agreement_countries_post)

X = complete_agreement_countries_post[['const', 'year', 'pop_density', 'gov_effectiveness']]
y = complete_agreement_countries_post['co2']

model = sm.OLS(y, X).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    co2   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     41.57
Date:                Fri, 22 Mar 2024   Prob (F-statistic):           3.94e-19
Time:                        20:33:23   Log-Likelihood:                -353.72
No. Observations:                 140   AIC:                             715.4
Df Residuals:                     136   BIC:                             727.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               161.8554    370.40

In [77]:
#REST OF THE WORLD CO2 REGRESSION ANALYSIS
rest_of_the_world_post = rest_of_the_world[(rest_of_the_world['year'] >= 2016) & (rest_of_the_world['year'] <= 2020)].copy()
rest_of_the_world_post['year'] = rest_of_the_world_post['year'].astype(int)

rest_of_the_world_post = rest_of_the_world_post.loc[:, ['country_name', 'year', 'co2', 'pop_density', 'gov_effectiveness']]
rest_of_the_world_post = rest_of_the_world_post.dropna(subset=['co2'])
rest_of_the_world_post['year'] = pd.to_numeric(rest_of_the_world_post['year'])

rest_of_the_world_post = sm.add_constant(rest_of_the_world_post)

X = rest_of_the_world_post[['const', 'year', 'pop_density', 'gov_effectiveness']] 
y = rest_of_the_world_post['co2']

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    co2   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     93.03
Date:                Fri, 22 Mar 2024   Prob (F-statistic):           1.58e-51
Time:                        20:33:23   Log-Likelihood:                -2253.0
No. Observations:                 790   AIC:                             4514.
Df Residuals:                     786   BIC:                             4533.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               163.7453    213.30

### The above regression results imply that there is no substantial difference between countries that signed the Paris Climate Accord and those that did not! We see that the point estimate for the coefficient on "year" in the first regression is -0.0785 which uses data for treat-signing countries, while it is -0.0790 for the second regression which uses data from the rest of the non-treaty-signing countries. This means that after controlling for population density and government effectiveness, each subsequent year post-treaty is associated with a 0.0785 reduction in Metric Tons of CO2 emissions for treaty-signing countries while each subsequent year is associated with a 0.079 reduction in Metric Tons of CO2 emissions for non-treaty signing countries! Thus, countries who signed the agreement were no better off at reducing their CO2 emissions than the rest of the world.

# Are developed countries who signed the Paris Climate Accord doing better relative to developing countries in reducing CO2 emissions?

In [78]:
developing_countries = complete_agreement_countries[complete_agreement_countries['econ_status'] == 'Developing']
developed_countries = complete_agreement_countries[complete_agreement_countries['econ_status'] == 'Developed']

In [79]:
complete_developing_countries_2016 = developing_countries[developing_countries['year'] == 2016]
complete_developing_countries_2020 = developing_countries[developing_countries['year'] == 2020]

merged_df = pd.merge(complete_developing_countries_2016, complete_developing_countries_2020, on=['country_name', 'country_code'], suffixes=('_2016', '_2020'))

merged_df['co2_percentage_change'] = ((merged_df['co2_2020'] - merged_df['co2_2016']) / merged_df['co2_2016']) * 100

result_df = merged_df[['country_name', 'country_code', 'co2_percentage_change']]

chart = alt.Chart(result_df).mark_bar().encode(
    x=alt.X('country_name:N', title='Country', sort='y'),  # Sort countries by co2_difference in descending order
    y=alt.Y('co2_percentage_change:Q', title='% Change in CO2 from 2016 to 2020'),
    color=alt.condition(
        alt.datum.co2_percentage_change > 0,
        alt.value('red'),  # Positive difference
        alt.value('green')  # Negative difference
    )
).properties(
    width=1000,
    height=500,
    title='Fig 5. % Change in CO2 Emissions (2016-2020) for Developing Countries Post-Paris Climate Accord'
)

# Add horizontal line at y=0
horizontal_line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='black').encode(y='y')

# Combine chart and horizontal line
combined_chart = (chart + horizontal_line).configure_title(
    fontSize=25  # 
)

combined_chart

alt.LayerChart(...)

In [80]:
complete_developed_countries_2016 = developed_countries[developed_countries['year'] == 2016]
complete_developed_countries_2020 = developed_countries[developed_countries['year'] == 2020]

merged_df = pd.merge(complete_developed_countries_2016, complete_developed_countries_2020, on=['country_name', 'country_code'], suffixes=('_2016', '_2020'))

merged_df['co2_percentage_change'] = ((merged_df['co2_2020'] - merged_df['co2_2016']) / merged_df['co2_2016']) * 100

result_df = merged_df[['country_name', 'country_code', 'co2_percentage_change']]

chart = alt.Chart(result_df).mark_bar().encode(
    x=alt.X('country_name:N', title='Country', sort='y'),  # Sort countries by co2_difference in descending order
    y=alt.Y('co2_percentage_change:Q', title='% Change in CO2 from 2016 to 2020'),
    color=alt.condition(
        alt.datum.co2_percentage_change > 0,
        alt.value('red'),  # Positive difference
        alt.value('green')  # Negative difference
    )
).properties(
    width=700,
    height=300,
    title='Fig 6. % Change in CO2 Emissions (2016-2020) for Developed Countries Post-Paris Climate Accord'
)

# Add horizontal line at y=0
horizontal_line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='black').encode(y='y')

# Combine chart and horizontal line
combined_chart = (chart + horizontal_line).configure_title(
    fontSize=20  # 
)

combined_chart

alt.LayerChart(...)

## A simple chart of % change in CO2 emissions between 2016 and 2020 for Developed and Developing Countries implies that Developed Countries did much better at following through on the Paris Climate Accord than Developing Countries. However, we also find that some developing countries were able to significantly reduce their CO2 emissions from 2016 to 2020. Although this is the case, questions of climate responsibility are relevant to this analysis: are developed countries only able to meet their climate targets because they are more well-off and can afford to reduce their CO2 emissions in the short-term while developing countries cannot afford to do so?

In [81]:
complete_agreement_countries_post = developed_countries 
complete_agreement_countries_post = complete_agreement_countries_post.loc[:, ['country_name', 'year', 'co2', 'pop_density', 'gov_effectiveness']]

complete_agreement_countries_post = complete_agreement_countries_post[(complete_agreement_countries_post['year'] >= 2016) & (complete_agreement_countries_post['year'] <= 2020)].copy()
complete_agreement_countries_post['year'] = complete_agreement_countries_post['year'].astype(int)

complete_agreement_countries_post = complete_agreement_countries_post.dropna(subset=['co2'])
complete_agreement_countries_post['year'] = pd.to_numeric(complete_agreement_countries_post['year'])
complete_agreement_countries_post
complete_agreement_countries_post = sm.add_constant(complete_agreement_countries_post)

X = complete_agreement_countries_post[['const', 'year', 'pop_density', 'gov_effectiveness']]
y = complete_agreement_countries_post['co2']

model = sm.OLS(y, X).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    co2   R-squared:                       0.321
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     4.878
Date:                Fri, 22 Mar 2024   Prob (F-statistic):            0.00682
Time:                        20:33:23   Log-Likelihood:                -90.548
No. Observations:                  35   AIC:                             189.1
Df Residuals:                      31   BIC:                             195.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               601.9551    824.24

In [82]:
complete_agreement_countries_post = developing_countries 
complete_agreement_countries_post = complete_agreement_countries_post.loc[:, ['country_name', 'year', 'co2', 'pop_density', 'gov_effectiveness']]

complete_agreement_countries_post = complete_agreement_countries_post[(complete_agreement_countries_post['year'] >= 2016) & (complete_agreement_countries_post['year'] <= 2020)].copy()
complete_agreement_countries_post['year'] = complete_agreement_countries_post['year'].astype(int)

complete_agreement_countries_post = complete_agreement_countries_post.dropna(subset=['co2'])
complete_agreement_countries_post['year'] = pd.to_numeric(complete_agreement_countries_post['year'])
complete_agreement_countries_post
complete_agreement_countries_post = sm.add_constant(complete_agreement_countries_post)

X = complete_agreement_countries_post[['const', 'year', 'pop_density', 'gov_effectiveness']]
y = complete_agreement_countries_post['co2']

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    co2   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     12.35
Date:                Fri, 22 Mar 2024   Prob (F-statistic):           6.01e-07
Time:                        20:33:23   Log-Likelihood:                -233.45
No. Observations:                 105   AIC:                             474.9
Df Residuals:                     101   BIC:                             485.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                22.4992    317.40

## Our analysis is substantiated by another set of linear regressions which uses data on 4 years post-treaty. We find that the point estimate for the coefficient on year is -0.2863 for developed countries and -0.0092 for developing countries. This implies that on average after controlling for population density and government effectiveness, developed countries that signed the Paris Climate Accord reduce their emissions by -0.2863 Metric Tons of CO2  on average, while developing countries that signed the Paris Climate Accord are only able to reduce their emissions by -0.2863 Metric Tons of CO2 on average. 

# From our analysis, we find answers to our research questions.
## 1. Do countries follow through on their climate agreements? The answer is that it depends on the country. Figure 3 shows which countries were the most/least effective at reducing their CO2 emissions from 2016 to 2020 (post-treaty).
## 2. Is government effectiveness for treaty-signing countries related to a reduction in CO2 emissions? Yes. We see in Figure 4 that countries with high government effectiveness also had proportional reductions in their CO2 emissions.
## 3. Were treaty-signing countries any more effective than non-treaty-signing countries at reducing CO2 emissions? No. We find from regression specifications 1 and 2 that treaty-singing countries reduced their CO2 emissions by as much as non-treaty signing countries in the 4 years post-treaty.
## 4. Are there differences in post-treaty commitment to CO2 emission reduction between developed and developing countries? Yes. From Fig. 5 & 6 and regression specifications 3 and 4, we find that developed countries and significantly more effective at decreasing their CO2 emissions than developing countries.

# Overall, while climate agreements are important in unifying efforts to reach specific climate metrics like reduction in CO2 emissions, it is important to consider factors like government effectiveness and heterogeneity in country characteristics such as development when evaluating country follow-through on climate agreements. In particular, we find that looking at the performance of non-treaty-signing countries is highly imformative in evaluating the performance of treaty-signing countries.